In [2]:
from datasets import load_dataset
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# ContextBasedQuestions = pd.read_csv('../Data/CleanedDatasets/ContextBasedQuestions.csv')
# ContextBasedQuestions_training = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions_training.csv")
# ContextBasedQuestions_testing = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions_testing.csv")
# ContextBasedQuestions_validation = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions_validation.csv")
# conversationalDatasetWithDomain = pd.read_csv('../Data/CleanedDatasets/conversationalDatasetWithDomain.csv')
# Greetings = pd.read_csv('../Data/CleanedDatasets/Greetings.csv')
# sentence_correction = pd.read_csv('../Data/CleanedDatasets/sentence_correction.csv')
# Summary_test = pd.read_csv('../Data/CleanedDatasets/Summary_testing.csv')
# Summary_training = pd.read_csv('../Data/CleanedDatasets/Summary_training.csv')
# Summary_validation = pd.read_csv('../Data/CleanedDatasets/Summary_validation.csv')
paraphrase = pd.read_csv("../Data/CleanedDatasets/paraphrased.csv")

In [32]:
X_train, X_test, y_train, y_test = train_test_split(paraphrase["agent_1"],paraphrase["agent_2"], test_size=0.2, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test,y_test, test_size=0.5, random_state=42)
dataset_training = pd.DataFrame()
dataset_training["agent_1"] = X_train; dataset_training["agent_2"] = y_train 
dataset_training.reset_index(inplace = True)
dataset_testing = pd.DataFrame()
dataset_testing["agent_1"] = X_test; dataset_testing["agent_2"] = y_test 
dataset_testing.reset_index(inplace = True)
dataset_validation = pd.DataFrame()
dataset_validation["agent_1"] = X_valid; dataset_validation["agent_2"] = y_valid
dataset_validation.reset_index(inplace = True)
dataset_training.to_csv("../Data/CleanedDatasets/paraphrase_training.csv")
dataset_testing.to_csv("../Data/CleanedDatasets/paraphrase_testing.csv")
dataset_validation.to_csv("../Data/CleanedDatasets/paraphrase_validation.csv")

In [18]:
dataset_validation

,index,agent_1,agent_2
0,302112,<paraphrase>Two more were established by Grove...,"Shortly after World War II, two additional nat..."
1,16327187,<paraphrase>The inside of St Mark's Basilica i...,The St Mark's Basilica in Venice is decorated ...
2,9100195,Can you rephrase this sentence in different wo...,"Sure sir, here is the reinterpreted form of th..."
3,19020598,Can you restate this sentence differently?From...,"Sure sir, here’s the rephrased rendition of it..."
4,8703749,Please rewrite this sentence in your own way.C...,"Yes sir, here is the reformulated sentence you..."
...,...,...,...
2908858,12099737,<paraphrase>What are the benefits and drawback...,What are the positive and negative aspects of ...
2908859,5838151,<paraphrase>It carried a radar reflector and c...,The airborne apparatus bore a radar mirror and...
2908860,16160027,<paraphrase>Among the countries with significa...,"The Philippines, Indonesia, Mexico, Italy, Ice..."
2908861,19726418,Rewrite this sentence with your own wording.Wo...,"Sure sir, here’s the reinterpreted version of ..."


In [9]:
dataset = Summary_training._append(ContextBasedQuestions)
dataset.reset_index(inplace=True)
dataset = dataset[["agent_1","agent_2"]]

In [10]:
dataset

,agent_1,agent_2
0,agent_1:Could you write a brief summary of thi...,agent_2:The course offered a solid foundation ...
1,agent_1:Can you offer a concise summary of thi...,agent_2:The introductory pattern recognition c...
2,agent_1:Can you summarize the key points of th...,agent_2:The introductory pattern recognition c...
3,agent_1:Can you provide a brief synopsis of th...,agent_2:The Pattern Recognition course struggl...
4,agent_1:Could you outline the main ideas of th...,agent_2:The Pattern Recognition course was an ...
...,...,...
3532827,<context> Creating ThemesThe game room is wher...,agent_2:Here are two images illustrating how t...
3532828,<context> Creating ThemesThe game room is wher...,agent_2:Notice that every even gameboard units...
3532829,<context> Creating ThemesThe game room is wher...,agent_2:The background image in this example c...
3532830,<context> Creating ThemesThe game room is wher...,agent_2:The client software has 8 chess sets t...


In [4]:
Summary_training

,Unnamed: 0,agent_1,agent_2
0,0,agent_1:Could you write a brief summary of thi...,agent_2:The course offered a solid foundation ...
1,1,agent_1:Can you offer a concise summary of thi...,agent_2:The introductory pattern recognition c...
2,2,agent_1:Can you summarize the key points of th...,agent_2:The introductory pattern recognition c...
3,3,agent_1:Can you provide a brief synopsis of th...,agent_2:The Pattern Recognition course struggl...
4,4,agent_1:Could you outline the main ideas of th...,agent_2:The Pattern Recognition course was an ...
...,...,...,...
2131996,2131996,agent_1:Would you summarize this article in br...,agent_2:Plans to expand Anfield have been subm...
2131997,2131997,agent_1:<summary> would be determined by certa...,agent_2:Plans to expand Anfield have been subm...
2131998,2131998,agent_1:Please summarize the main ideas of thi...,agent_2:Consumers get constant pitches for her...
2131999,2131999,agent_1:<summary>t be pre-approved by the Food...,agent_2:Consumers get constant pitches for her...


In [9]:
X_train, X_test, y_train, y_test = train_test_split(conversationalDatasetWithDomain["agent_1"],conversationalDatasetWithDomain["agent_2"], test_size=0.2, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test,y_test, test_size=0.5, random_state=42)

In [10]:
dataset = pd.DataFrame()
dataset["agent_1"] = X_train; dataset["agent_2"] = y_train
dataset = dataset.sample(frac = 1)

In [11]:
dataset = dataset[-400000:]
merge_df = dataset[["agent_1","agent_2"]]._append(Greetings[["agent_1","agent_2"]])
merge_df.reset_index(inplace=True)
merge_df.drop("index",axis = 1,inplace = True)
merge_df = merge_df.sample(frac = 1)
# merge_df.to_csv("../Data/CleanedDatasets/Greetings_plus_conversationalDataset.csv")

In [17]:
merge_df

,agent_1,agent_2
144053,"agent_1:Hey Sky,Do you think children watch to...",{<Content>: I only allow my child to watch up...
63268,"agent_1:Hello Sky,Will you answer my questions...","{<Content>: Hi Akshay Sir,To solve this proble..."
246139,"agent_1:Sky, What is the most influential fact...",{<Content>: Monetary policies controlled clos...
350721,"agent_1:Hello Sky, I need help plotting a char...",{<Content>: You can use many different progra...
385494,"agent_1:Sky, Pizza party for 15- what do I dou...",{<Content>: I’m going to catch some downvotes ...
...,...,...
208286,"agent_1:Hello Sky,Will you answer my questions...","{<Content>: Hi Akshay Sir,The binomial coeffic..."
64698,"agent_1:Hello Sky, Economics are based heavily...",{<Content>: There are a number of strategic d...
107609,"agent_1:Hey Sky,I want you to act as a doctor....","{<Content>: Hi Akshay Sir,hi and thanks for th..."
55324,"agent_1:Sky, How to recognize my emotions and ...",{<Content>: Understanding emotional issues an...


In [3]:
ContextBasedQuestions

,Unnamed: 0,agent_1,context,agent_2
0,0,why do the healthiest of foods cause bad gas?,Actually it is because most of people eat tha...,These foods contain certain sugars that our bo...
1,1,"In layman's terms, what would actually happen ...",The decay of false vacuum is a fascinating to...,I'd answer the question but the default reddit...
2,2,"When I move at speed through the air, it's col...",The spacecraft re-enters through the atmosphe...,There are (at least) 3 different competing mec...
3,3,what is the big deal about russia invading ukr...,National Sovereignty is a very big deal so an...,Essentially the events in Crimea threaten to d...
4,4,why are there different types of helmets if yo...,Most helmets are pretty similar in basic cons...,To have as little protection as is safe for th...
...,...,...,...,...
1802138,1802138,Question: Can you convert this paragraph into ...,Creating ThemesThe game room is where you are ...,Here are two images illustrating how the game ...
1802139,1802139,Game room colors 1,Creating ThemesThe game room is where you are ...,Notice that every even gameboard units are hid...
1802140,1802140,Game room colors 2,Creating ThemesThe game room is where you are ...,The background image in this example could be ...
1802141,1802141,Just load the MMO board game client software t...,Creating ThemesThe game room is where you are ...,The client software has 8 chess sets to choose...


In [15]:
conversationalDatasetWithDomain

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,agent_1,old_reply,emotion,Keywords,domain,agent_2
1860234,1860234,1860234,1860234,"agent_1:Hello Sky,agent_2:Hello Akshay Sir,age...","Hi Akshay Sir,Judy uses 10 pencils over a 5-da...",6,'Akshay Sir',3,"{<Content>: Hi Akshay Sir,Judy uses 10 pencils..."
2169634,2169634,2169634,2169634,"agent_1:Sky,agent_2:Hello Akshay Sir,agent_1:M...","Hi Akshay Sir,The symptoms and severity of Mil...",6,"'Milroy disease', 'lymphangiosarcoma",2,"{<Content>: Hi Akshay Sir,The symptoms and sev..."
21015,21015,21015,21015,"agent_1:Hi Sky, I am looking for the most suit...","Yes, there are many different international c...",6,"International communication, ITsecurity, Cyber...",6,"{<Content>: Yes, there are many different int..."
1491758,1491758,1491758,1491758,"agent_1:Hello Sky,Rewrite the following senten...","Hello Akshay Sir,The bus arrived late, so I ha...",6,"Rewrite, comma, independent clause",3,"{<Content>: Hello Akshay Sir,The bus arrived l..."
1701588,1701588,1701588,1701588,"agent_1:Hello Sky,I want you to act as a mathe...","Hello Akshay Sir,If two CDs are each 1.5 hours...",6,'Akshay Sir',7,"{<Content>: Hello Akshay Sir,If two CDs are ea..."
...,...,...,...,...,...,...,...,...,...
561937,561937,561937,561937,"agent_1:Hello Sky, How can I increase my searc...","1. Ensure all content is high quality, unique...",6,"Search engine ranking, Content optimization, S...",4,{<Content>: 1. Ensure all content is high qua...
1964940,1964940,1964940,1964940,"agent_1:Hello Sky,Please answer this question ...","Hello Akshay Sir,The probable diagnosis is cho...",6,"choriocarcinoma, imaging tests, medical treatment",2,"{<Content>: Hello Akshay Sir,The probable diag..."
1049837,1049837,1049837,1049837,"agent_1:Hey Sky, What kind of foods should I f...","Turtles are omnivores, and there’s really no ...",6,"pet turtle, foods, omnivores",7,"{<Content>: Turtles are omnivores, and there’..."
526322,526322,526322,526322,"agent_1:Hello Sky, Select the best match for e...",(d),6,"Agent_1, Hello Sky, List Selection",7,"{<Content>: (d),<Emotion>: 6,<Domain>: 7}"


In [17]:
type(X_train)

pandas.core.series.Series

In [3]:
Greetings

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,agent_1,agent_2,emotion,domain
0,0,0,0,0,0,Hi,{<Content>: Hello there. Tell me how are you f...,6,6
1,1,1,1,1,1,Hi,{<Content>: Hi there. What brings you here tod...,6,6
2,2,2,2,2,2,Hi,{<Content>: Hi there. How are you feeling toda...,6,6
3,3,3,3,3,3,Hi,{<Content>: Great to see you. How do you feel ...,6,6
4,4,4,4,4,4,Hi,{<Content>: Hello there. Glad to see you're ba...,6,6
...,...,...,...,...,...,...,...,...,...
713,713,713,713,717,717,How do I know if I'm unwell?,"{<Content>: If your beliefs , thoughts , feeli...",13,2
714,714,714,714,718,718,How can I maintain social connections? What if...,{<Content>: A lot of people are alone right no...,6,8
715,715,715,715,719,719,What's the difference between anxiety and stress?,{<Content>: Stress and anxiety are often used ...,6,2
716,716,716,716,720,720,What's the difference between sadness and depr...,{<Content>: Sadness is a normal reaction to a ...,1,2


In [6]:
sentence_correction

,Unnamed: 0,agent_1,agent_2
0,0,"<reconstruct>strength. 2,000 of comined per th...",It is estimated that the Turkish 27th and 57th...
1,1,"<reconstruct>Second at acros BaJh School, 9.13...","Second across was Kingswood School, Bath at 9.13."
2,2,<reconstruct>The player who controls the omst ...,The player who controls the most citadels afte...
3,3,"<reconstruct>ae Nairobi, in Entebbe, Tanzania....",The airline operates scheduled passenger servi...
4,4,<reconstruct>(9) Quayle Campaign Rally from Se...,"(9) Quayle Campaign Rally from September 27, 1988"
...,...,...,...
1948015,1948015,Could you please check for grammar errors in t...,"The field, legend, and border are all retroref..."
1948016,1948016,"Identify grammar mistakes in this sentence, pl...","As of 1882, Partridge was responsible for havi..."
1948017,1948017,Can you verify the grammatical accuracy of thi...,"Peabody, his brother Charles Jones Peabody and..."
1948018,1948018,Please verify there are no grammar mistakes in...,"A ""Final Edition"" (FE) trim was offered after ..."


In [7]:
Summary_test

,Unnamed: 0,agent_1,agent_2
0,0,Could you sum up this article in detail for me...,The pattern recognition course was initially p...
1,1,Can you write a summary of this article in ful...,"The course on pattern recognition, initially p..."
2,2,Could you condense this paragraph into a depth...,The course on pattern recognition concepts cou...
3,3,Can you summarize the key points of this parag...,The course on pattern recognition concepts cou...
4,4,Would you mind summarizing the key points of t...,The course on pattern recognition techniques p...
...,...,...,...
55273,55273,"was getting a good read on the serve and, whe...",Britain's Andy Murray is through to the Monte ...
55274,55274,Would you mind giving a quick summary of this ...,A forensics expert has swung a cricket bat at ...
55275,55275,ide.\nMr Roux's plan seems to include discredi...,A forensics expert has swung a cricket bat at ...
55276,55276,hat a mark on the bottom of the door showed wh...,A forensics expert has swung a cricket bat at ...


In [8]:
Summary_training

,Unnamed: 0,agent_1,agent_2
0,0,Can you briefly summarize this article?'This i...,The course offered a solid foundation for expl...
1,1,Can you briefly outline the main points of thi...,The introductory pattern recognition course ai...
2,2,Could you summarize the main ideas of this doc...,The introductory pattern recognition course fa...
3,3,Could you sum up the key ideas of this article...,The Pattern Recognition course struggled to en...
4,4,Would you mind summarizing the key points of t...,The Pattern Recognition course was an excellen...
...,...,...,...
613062,613062,Could you sum up this article in short for me?...,Erik Sviatchenko is adamant that Celtic will p...
613063,613063,"o into it thinking it is going to be easy.\n""T...",Erik Sviatchenko is adamant that Celtic will p...
613064,613064,Would you summarize this document in short for...,People have spent a large part of this season ...
613065,613065,ition has made a big difference to the team.\n...,People have spent a large part of this season ...


In [9]:
Summary_validation

,Unnamed: 0,agent_1,agent_2
0,0,Please provide an overview of this document in...,The course offered a solid foundation for expl...
1,1,Would you provide an outline of this document?...,The introductory pattern recognition course fa...
2,2,Can you briefly outline this document in depth...,The pattern recognition course provided a soli...
3,3,Would you mind giving an overview of this arti...,The introductory course on pattern recognition...
4,4,Would you summarize the content of this docume...,"The pattern recognition course, aimed at provi..."
...,...,...,...
53721,53721,Can you offer a concise summary of this paragr...,The government in Brazil says it will send fed...
53722,53722,Would you mind providing an overview of this d...,The US Department of Justice (DoJ) has approve...
53723,53723,Could you sum up this document in short for me...,Britain's Andy Murray will face world number o...
53724,53724,t Murray's team along with Vallverdu last Nove...,Britain's Andy Murray will face world number o...


In [30]:
# paraphrase["agent_2"] = "agent_2:"+paraphrase["agent_2"]
# paraphrase["agent_1"] = "agent_1:"+paraphrase["agent_1"]

In [31]:
paraphrase

,Unnamed: 0,agent_1,agent_2
0,0,agent_1:<paraphrase>In 1928 he submitted a pat...,"agent_2:In the year 1928, he filed an applicat..."
1,1,agent_1:<paraphrase>On 9 March the 1st Infantr...,"agent_2:On the 9th of March, the first infantr..."
2,2,"agent_1:<paraphrase>Thus, Dimant claims """"Jubi...","agent_2:According to Dimant, ""Jubilees"" resolv..."
3,3,agent_1:<paraphrase>Harris also said that libe...,"agent_2:According to Harris, Liberalism has be..."
4,4,agent_1:<paraphrase>The total inflow of energy...,agent_2:The conservation of energy principle d...
...,...,...,...
29088623,29088623,agent_1:Can you paraphrase this article differ...,"agent_2:Certainly sir, here is the rephrased f..."
29088624,29088624,agent_1:Can you restate this article using you...,"agent_2:Sure sir, here is the paraphrased stat..."
29088625,29088625,agent_1:Can you rewrite this sentence in your ...,"agent_2:Yes sir, here is the reformulated vers..."
29088626,29088626,agent_1:Can you put this sentence into differe...,"agent_2:Sure thing sir, here is the restated f..."


### Testing the Model

**18/09/2024**   
Average Rouge Score (Overlap of unigrams) :  0.477, 
Average Rouge Score (Longest Sequence) :  0.40

**25/09/2024**   
Average Rouge Score (Overlap of unigrams) :  0.4790
Average Rouge Score (Longest Sequence) :  0.398

**25/09/2024 - 1**   
Average Rouge Score (Overlap of unigrams) :  0.4694
Average Rouge Score (Longest Sequence) :  0.38905

**25/09/2024 - 2**   
Average Rouge Score (Overlap of unigrams) :  0.4709
Average Rouge Score (Longest Sequence) :  0.382
5


In [2]:
dataset = pd.read_csv("../Data/CleanedDatasets/Greetings_plus_conversationalDataset.csv")
X_train, X_test, y_train, y_test = train_test_split(dataset["agent_1"],dataset["agent_2"], test_size=0.2, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test,y_test, test_size=0.5, random_state=42)

In [20]:
# Summary_test = pd.read_csv('../Data/CleanedDatasets/Summary_testing.csv')
# Summary_test = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions_testing.csv")
Summary_test = pd.read_csv("../Data/CleanedDatasets/paraphrase_testing.csv")

In [21]:
Summary_test = Summary_test.sample(frac = 1)
X_test = Summary_test["agent_1"]
y_test = Summary_test["agent_2"]

In [10]:
import torch
torch.cuda.empty_cache()
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA available: True
CUDA version: 12.1
Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4090


In [12]:
model_name = '../Saved_Models/Sky/fine-tuned-bert-sentiment_2024_10_05_0'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
def rouge_calculate(reference,candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    
    return scores['rouge1'].fmeasure,scores['rougeL'].fmeasure

In [22]:
rogue_score_1 = []
rogue_score_L = []
for counter in range(0,3):
    # print(f'\rProgress: {counter}/{100}', end='', flush=True)
    print("Question : {}\n".format(X_test.tolist()[counter]))
    print()
    # testDict["Question"].append(X_test.tolist()[counter])
    input_text = X_test.tolist()[counter]
    # input_text = "<context> The rapid rise of artificial intelligence (AI) has transformed industries across the globe. Companies are now leveraging AI to streamline processes, enhance decision-making, and improve customer experiences. AI-powered tools are used in a variety of applications, from automating mundane tasks to providing personalized recommendations in real time. However, as AI continues to evolve, concerns regarding its ethical use are becoming increasingly important. Issues such as data privacy, bias in AI algorithms, and job displacement are at the forefront of discussions. Companies and policymakers must work together to ensure that AI systems are transparent, fair, and accountable. Additionally, the responsibility of training AI models with diverse and representative data cannot be overlooked. With appropriate safeguards and regulations, AI can be harnessed for positive societal impact, but it requires a concerted effort from all stakeholders. As AI becomes more integrated into everyday life, understanding its implications and addressing potential risks will be key to realizing its full potential.agent_1:How might AI affect industries in the future as it becomes more integrated into everyday life?"
    input_ids = tokenizer.encode(input_text, return_tensors='pt',max_length=512, truncation=True).to(device)
    output = model.generate(
        input_ids, 
        max_length=256, 
        num_beams=5, 
        early_stopping=True, 
        no_repeat_ngram_size=2,  # Prevent repeating n-grams
        num_return_sequences=1,  # Number of sequences to return
        temperature=0.7,  # Sampling temperature
        top_k=50,  # Top-K sampling
        top_p=0.9  # Top-p (nucleus) sampling
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Predicted Answer:\n", response)
    print()
    print("True: ",y_test.tolist()[counter])
    print()
    # testDict["Predicted"].append(response)
    # testDict["True"].append(y_test.tolist()[counter])
    r1,r2 = rouge_calculate(y_test.tolist()[counter],response)
    rogue_score_1.append(r1)
    rogue_score_L.append(r2)
print()
print("Average Rouge Score (Overlap of unigrams) : ",np.mean(rogue_score_1))
print("Average Rouge Score (Longest Sequence) : ",np.mean(rogue_score_L))

Question : agent_1:<paraphrase>It seems a one-drop rule in reverse.


Predicted Answer:
 agent_2:It appears to be a one-drop rule in reverse.

True:  agent_2:It appears to be a reversal of the one-drop rule.

Question : agent_1:<paraphrase>The world's population is becoming more intricate, and analyzing more groups would necessitate more markers for the same outcome.


Predicted Answer:
 agent_2:The complexity of the world's population is increasing, and analyzing more groups would necessitate more markers for the same result.

True:  agent_2:The global population is becoming more intricate, and studying an increasing number of groups would necessitate more markers to achieve the same outcome.

Question : agent_1:Can you paraphrase this article differently?"Due to the lack of familiarity of the Barsoom works among the general public, "John Carter" seemed to be imitative of the very things it was influenced by."


Predicted Answer:
 agent_2:Sure sir, here’s the rephrased rendition of it